# Instrumentbræt for COVID-19 i Danmark

**Dette dokument viser forskellige nøgletal for COVID-19 i Danmark baseret på daglige tal fra Statens Serum Institut.**

*Koden til denne notebook findes på GitHub, se [github.com COVID-19-dk](https://github.com/covid-19-sim/covid-19-dk)*

## Datagrundlag

Data er indhentet fra Sundhedsstyrelsens hjemmeside for
[COVID-19 tal og overvågning](https://www.sst.dk/da/corona/tal-og-overvaagning).

Læs mere om datasættet på https://github.com/covid-19-sim/covid-19.sst.dk

In [ ]:
import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
import seaborn as sns
import scipy.optimize
from scipy.ndimage.filters import gaussian_filter1d

plt.rcParams['figure.figsize'] = [15, 10]
plt.rcParams['figure.facecolor'] = '0.9'

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/covid-19-sim/covid-19.sst.dk/master/total-covid-19-dk.csv',
                 parse_dates=['Date'])

# Kontroller at data har det forventede format
for c in ['Date', 'Tested', 'Confirmed', 'Total tested', 'Total confirmed', 'Hospitalised', 'ICU', 'ICU-vent', 'Deaths', 'Total deaths']:
    assert c in df.columns

In [ ]:
# Epidemistarten er den første dato
EPIDEMI_START = pd.Timestamp("2020-03-03")

df['Day'] = df['Date'] - EPIDEMI_START
df['Day N'] = df['Day'].dt.days

#
# Udvælg data vi interesserer os for
OVERSKRIFTER = {'Tested': 'Prøver',
                'Confirmed': 'Positive prøver',
                'Total tested': 'Prøver i alt',
                'Total confirmed': 'Positive prøver i alt',
                'Hospitalised': 'Indlagte',
                'ICU': 'Intensiv',
                'ICU-vent': 'Intensiv med respirator',
                'Total deaths': 'Døde'}

kolonner = list(OVERSKRIFTER.keys())

# Hvis ikke vi har observationen, så drop rækken
data = df[['Day N'] + kolonner]

In [ ]:
fig, axes = plt.subplots(nrows=2,ncols=int(np.ceil(len(kolonner)/2.)), sharey=True)
for i, kolonne in enumerate(kolonner):
    ax = axes.flat[i]
    ax.semilogy(data['Day N'], data[kolonne])
    ax.set_xlabel('Dag')
    ax.set_ylabel('Antal')
    ax.set_title(f'{OVERSKRIFTER[kolonne]}')
    ax.grid()

## Prøvetagning

Status på prøvetagningen kan vises på flere måder.

1. Første nøgletal er hvor mange prøver der er taget.
2. Andet nøgletal er forholdet mellem antal positive prøver i alt og total prøvetagning.
3. Tredie nøgletal er andelen af positive prøver på dagsbasis. Når prøvetagningen bliver bredere konvergerer det mod udbredelsen af sygdommen.

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3)

ax = axes[0]
ax.plot(df['Day N'], df['Total tested'], marker='o')
ax.grid()
ax.set_title('Antal prøver i alt')
ax.set_xlabel('Dag')
ax.set_ylabel(OVERSKRIFTER['Total tested'])

ax = axes[1]
ax.loglog(df['Total tested'], df['Total confirmed'], marker='o')
ax.set_xlabel(OVERSKRIFTER['Total tested'])
ax.set_ylabel(OVERSKRIFTER['Total confirmed'])
ax.grid()
ax.set_title('Prøvetagning fremdrift')

ax = axes[2]
andel_positive = df['Confirmed']/df['Tested']
andel_positive_middel = andel_positive.rolling(7).mean()
ax.scatter(df['Day N'], andel_positive, marker='o', label='Andel positive')
ax.plot(df['Day N'], andel_positive_middel, '--', label='7-dages middel')
ax.grid()
ax.set_xlabel('Dag')
ax.set_ylabel('Andel positive')
ax.set_title('Andel positive prøver på dagsbasis')
ax.get_yaxis().set_major_formatter(mticker.PercentFormatter(xmax=1))
ax.legend(loc='upper right')
fig.tight_layout(pad=3)

## Dødelighed

Lad os se, hvor slemt det står til.

For at have noget at sammenligne med, anvender vi 2018-dødsårsager fra Danmarks Statistik.
Kilde: https://www.dst.dk/da/Statistik/emner/befolkning-og-valg/doedsfald-og-middellevetid/doedsfald

In [ ]:
DANMARKS_STATISTIK_DOEDSAARSAGER_2018 = {
    'I alt': 55_232,
    'Hjertesygdomme': 6_627,
    'Kræftsygdomme': 15_342,
    'Karsygdomme i hjernen': 3_190,
    'Lungebetændelse og influenza': 2_103,
    'Bronkitis og astma': 3_789,
    'Sukkersyge': 1_358,
    'Mentale lidelser': 3_711,
    'Motorkøretøjsulykker': 186,
    'Andre ulykker': 1_221,
    'Selvmord':	584,
    'Øvrige dødsårsager': 16_741,
    'Uoplyst dødsårsag': 380
}

In [ ]:
# Først skal vi bruge nogle konverteringsfunktioner mellem dage og datoer
# så vi kan vise både dato og dage siden epidemistart på to x-akser.

def date_to_day(x):
    """
    x is in matplotlib datenums, so they are floats.
    """
    y = x - mdates.date2num(EPIDEMI_START)
    return y


def day_to_date(x):
    """
    return a matplotlib datenum (x is days since start date)
    """
    y = x + mdates.date2num(EPIDEMI_START)
    return y

In [ ]:
datoer = df['Date']
dage = df['Day N']
doede = df['Total deaths']

fig, ax = plt.subplots()
fig.autofmt_xdate()
ax.set_yscale('log')
ax.grid(True)
ax.plot(datoer, doede, marker='o')
ax.xaxis_date()
ax.set_xlabel('Dato')
ax.get_xaxis().set_major_locator(mdates.WeekdayLocator(mdates.MONDAY))
ax.get_xaxis().set_minor_locator(mdates.DayLocator())
ax.set_ylabel(OVERSKRIFTER['Total deaths'])

dage_x = ax.secondary_xaxis('top', functions=(date_to_day, day_to_date))
dage_x.set_xlabel('Dage siden epidemistart')

fig.suptitle('Dødsfald under epidemien')

Vi kan sammenligne med andre if dødsårsager:
De stiplede linier er gennemsnitligt antal dagelige døde fra 2018-dødsårsagerne lineært
fremskrevet over epidemiperioden.

In [ ]:
fig, ax = plt.subplots()

for doedsaarsag in sorted(DANMARKS_STATISTIK_DOEDSAARSAGER_2018, key=DANMARKS_STATISTIK_DOEDSAARSAGER_2018.get, reverse=True):
    antal = DANMARKS_STATISTIK_DOEDSAARSAGER_2018[doedsaarsag]
    #ax.axhline(antal, ls='--', label=doedsaarsag)
    #ved at bruge plot får vi automatisk colour cycling på liniefarverne
    ax.plot(datoer, dage*antal/365., ls='--', label=doedsaarsag)

fig.autofmt_xdate()
ax.grid(True)
ax.plot(datoer, doede, marker='o', c='r', linewidth=3, label='COVID-19')
ax.xaxis_date()
ax.set_xlabel('Dato')
ax.get_xaxis().set_major_locator(mdates.WeekdayLocator(mdates.MONDAY))
ax.get_xaxis().set_minor_locator(mdates.DayLocator())
ax.set_ylabel(OVERSKRIFTER['Total deaths'])

dage_x = ax.secondary_xaxis('top', functions=(date_to_day, day_to_date))
dage_x.set_xlabel('Dage siden epidemistart')

fig.suptitle('Dødsfald COVID-19 og andre årsager')

ax.legend()

Andelen af COVID-19 dødsfald sammenlignet med den gennemsnitlige dødelighed for
andre årsager for samme periodelængde for 2018.

In [ ]:
dag_max = doede.idxmax()

labels_2018 = [k for k in DANMARKS_STATISTIK_DOEDSAARSAGER_2018.keys() if k != 'I alt']
sizes_2018 = np.array([DANMARKS_STATISTIK_DOEDSAARSAGER_2018[k] for k in labels_2018])
# Lineær fremskrivning af gns antal daglige døde til dag_max
sizes_pr_dag_max = [dag_max/365.*s for s in sizes_2018]

labels = ['COVID-19'] + labels_2018
sizes = [doede[dag_max]] + sizes_pr_dag_max

explode = np.zeros_like(sizes)
explode[0] = 0.1 # Træk ostestykket med COVID-19 lidt ud

fig, ax = plt.subplots()

ax.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%', shadow=True, startangle=90)
ax.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

fig.suptitle('Estimeret andel af dødsårsager i perioden')
plt.show()